# Retraining MeTA classification model

In this notebook, we guide you through training a random forest model with our data and your labeled data, and then using that model to make predictions on your unlabeled data.

In [1]:
#standard 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns

#model to use
from sklearn.ensemble import RandomForestClassifier

#model performance
from sklearn.metrics import confusion_matrix, f1_score, precision_recall_fscore_support
from sklearn.model_selection import cross_val_predict, cross_val_score, KFold, RandomizedSearchCV, train_test_split
from scipy.stats import randint as sp_randint
from time import time
#for easy visualization of model performance
import scikitplot as skplt

#importing model
import pickle 

#custom function for trim mean and std deviation
def trim_mean_std(data, frac=0.05):    
    mean = stats.trim_mean(data, frac)
    std = stats.tstd(data, limits=(frac * np.max(data), (1 - frac) * np.max(data)))
    return mean, std

#Dictionary to convert amino acid abreviations
aa_dict = {'ALA': 'A', 'CYS':'C', 'ASP': 'D', 'GLU':'E',
          'PHE': 'F', 'GLY': 'G', 'HIS': 'H', 'ILE': 'I',
          'LYS': 'K', 'LEU': 'L', 'MET': 'M', 'ASN': 'N',
          'GLN': 'Q', 'ARG': 'R', 'SER': 'S', 'THR': 'T',
          'VAL': 'V', 'TRP': 'W', 'TYR': 'Y'}

#Dictionary to bucket amino acids
condensed_dict = {'L': 'LAIVMH', 'A': 'LAIVMH', 'I': 'LAIVMH', 'V': 'LAIVMH', 'M': 'LAIVMH', 'H': 'LAIVMH',
                  'D': 'DEQ', 'E': 'DEQ', 'Q': 'DEQ', 
                  'S': 'STGRFN', 'T': 'STGRFN', 'G': 'STGRFN', 'R': 'STGRFN', 'F': 'STGRFN', 'N': 'STGRFN',
                  'K': 'K'}

#important column headings
nums = ['0%', '1%', '5%', '10%']
ratio_cols = ['1/0', '5/0', '10/0', '5/1', '10/1', '10/5']

## Importing data
First, we import our data.

In [2]:
#read our data
IGPS = pd.read_csv('data/PTP1B_all.csv')
PTP1B = pd.read_csv('data/IGPS_all.csv')

Save your labeled data set in the folder "data." In the cell bellow, change the name of the string assigned to `labeled_data_filename` from `'sample_labeled.csv'` to your **labeled data set**. Structure the table identically to that in `'sample_labeled.csv'`. 

In [3]:
### INSERT YOUR LABELED DATA PATH HERE ####
labeled_data_filename = 'data/sample_labeled.csv'
labeled_data = pd.read_csv(labeled_data_filename).dropna(axis=0)
labeled_data.columns = ['0%', '1%', '5%', '10%', 'AA'] 
labeled_data.head()

,0%,1%,5%,10%,AA
0,70817.0,48836.26190,35416.90963,13722.06797,A
1,56457.0,41255.86774,34030.53055,14549.18591,A
2,84293.0,70637.32580,51737.79568,24447.40255,A
3,37792.0,27637.72522,20032.70190,12014.13218,A
4,87872.0,68161.90034,47255.06422,34808.69365,C


In the cell bellow, change the name of the string assigned to `unlabeled_data_filename` from `'sample_unlabeled.csv'` to your **unlabeled data set**. Structure the table identically to that in `'sample_labeled.csv'`. 

In [4]:
### INSERT YOUR UNLABELED DATA PATH HERE ####
unlabeled_data_filename = 'data/sample_unlabeled.csv'
unlabeled_data = pd.read_csv(unlabeled_data_filename) 
unlabeled_data.columns = ['0%', '1%', '5%', '10%'] 
unlabeled_data['AA'] = ['no_label'] * unlabeled_data.shape[0]
unlabeled_data.head()

,0%,1%,5%,10%,AA
0,44878.0,35399.39127,28197.90367,27808.87681,no_label
1,45768.0,31829.72942,14369.01023,18841.69090,no_label
2,80528.0,59971.64002,50153.13583,33555.46352,no_label
3,43642.0,36039.34709,27277.08781,27101.01449,no_label
4,98323.0,63613.22351,9940.21114,12906.80585,no_label


If you loaded your data, all you should have to do is execute cells until you get your predictions.

In [5]:
your_protein = pd.concat((labeled_data, unlabeled_data) , axis=0, ignore_index=True)

In [6]:
def create_ratios(data):
    ratio_cols = ['1/0', '5/0', '10/0', '5/1', '10/1', '10/5']
    div_by_zero = data[['1%', '5%', '10%']].values / data[['0%']].values 
    div_by_one = data[['5%', '10%']].values / data[['1%']].values
    div_by_five = data[['10%']].values / data[['5%']].values
    ratios = np.concatenate((div_by_zero, div_by_one, div_by_five), axis=1)
    ratios_df = pd.DataFrame(ratios, columns=ratio_cols)
    return ratios_df

In [7]:
#normal z score function
def z_score(data):
    means = np.mean(data, axis=0)
    stds = np.std(data, axis=0)
    zeroed_data = np.subtract(data, means)
    scaled_data = np.divide(zeroed_data, stds)
    return scaled_data 

#trim z score function
def trim_log_z_score(data, frac=0.05):
    '''
    Args:
    data (NUMPY ARRAY!!): numpy array of data to be log transformed and scaled, not robust to negative numbers
    frac (float): how much to trim mean and standard deviation
    Returns:
    scaled_data (numpy array): log transformed and scaled data
    '''
    log_data = np.log(np.clip(data, 0.1, None))
    trim_means_and_std = np.apply_along_axis(func1d=trim_mean_std, axis=0, arr=log_data, frac=frac)
    means = trim_means_and_std[0,:].reshape(1,-1)
    stds = trim_means_and_std[1,:].reshape(1,-1)
    zeroed_data = np.subtract(log_data, means)
    scaled_data = np.divide(zeroed_data, stds)
    return scaled_data

def ft_eng_and_scale(data):
    nums = ['0%', '1%', '5%', '10%']
    ratio_cols = ['1/0', '5/0', '10/0', '5/1', '10/1', '10/5']
    ratios_df = create_ratios(data)
    log_z_signal = trim_log_z_score(data[nums].values)
    z_ratio = z_score(ratios_df.values)
    X = np.concatenate((log_z_signal, z_ratio), axis=1)
    X_df = pd.DataFrame(X, columns=(nums+ratio_cols))
    X_df['AA'] = data['AA']
    return X_df

In [8]:
#preprocessing data by protein
IGPS_processed = ft_eng_and_scale(IGPS)
PTP1B_processed = ft_eng_and_scale(PTP1B)
your_protein_processed = ft_eng_and_scale(your_protein)

In [9]:
#separating user data by labeled and unlabeled
unlabeled_processed = your_protein_processed.loc[your_protein_processed['AA']=='no_label']
labeled_processed = your_protein_processed.loc[your_protein_processed['AA']!='no_label']

In [10]:
#formatting data for modelling and removing unmodeled amino acids
model_data = pd.concat((IGPS_processed, PTP1B_processed, labeled_processed), axis=0)
X = model_data.drop(columns=['AA']).values
y = model_data['AA']

def create_filter(AA_excluded, y, reverse=False):
    all_labels = ['L','A', 'I', 'V', 'D', 'E', 'Q', 'S', 'T', 'F', 'G', 'H', 'R', 'K', 'C', 'N', 'M', 'W', 'Y' ]
    labels = [x for x in all_labels if x not in AA_excluded]
    my_filter = [True if aa not in AA_excluded else False for aa in y]
    if reverse == True:
        reverse_filter = [True if aa in AA_excluded else False for aa in y]
        return my_filter, labels, reverse_filter
    else:
        return my_filter, labels
    
AA_excluded = ['W', 'Y', 'C']
my_filter, labels, reverse_filter = create_filter(AA_excluded, y, reverse=True)
X_filtered = X[my_filter, :]
y_filtered = y[my_filter]

We'll use cross-validation to tune parameters.

In [11]:
# Utility function to report best scores
def report(results, n_top=3):
    all_params = []
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
            all_params.append(results['params'][candidate])
    return all_params

In [12]:
clf_init = RandomForestClassifier(n_estimators=150, random_state=0)
# specify parameters and distributions to sample from
param_dist = {"max_depth": [3, None],
              "max_features": sp_randint(1, 10),
              "min_samples_split": sp_randint(2, 10),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run randomized search
n_iter_search = 25
random_search = RandomizedSearchCV(clf_init, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5, iid=False, random_state = 0)

start = time()
random_search.fit(X_filtered, y_filtered)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
all_params = report(random_search.cv_results_)

RandomizedSearchCV took 36.19 seconds for 25 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.732 (std: 0.240)
Parameters: {'bootstrap': True, 'criterion': 'gini', 'max_depth': None, 'max_features': 9, 'min_samples_split': 3}

Model with rank: 2
Mean validation score: 0.729 (std: 0.230)
Parameters: {'bootstrap': True, 'criterion': 'entropy', 'max_depth': None, 'max_features': 5, 'min_samples_split': 5}

Model with rank: 3
Mean validation score: 0.725 (std: 0.251)
Parameters: {'bootstrap': True, 'criterion': 'gini', 'max_depth': None, 'max_features': 7, 'min_samples_split': 2}



In [13]:
#select parameters
params = all_params[0]

In [14]:
clf = RandomForestClassifier(n_estimators=150, **params, random_state=0)
clf.fit(X, y)
unlabeled_preds = clf.predict(unlabeled_processed.drop(columns='AA').values)
bucketed_preds = [condensed_dict[i] for i in unlabeled_preds]
unlabeled_data = unlabeled_data.drop(columns=['AA'])
unlabeled_data['bucket'] = bucketed_preds
unlabeled_data.to_csv('output_data/retrained_output.csv')
unlabeled_data.head()

,0%,1%,5%,10%,bucket
0,44878.0,35399.39127,28197.90367,27808.87681,DEQ
1,45768.0,31829.72942,14369.01023,18841.69090,LAIVMH
2,80528.0,59971.64002,50153.13583,33555.46352,STGRFN
3,43642.0,36039.34709,27277.08781,27101.01449,DEQ
4,98323.0,63613.22351,9940.21114,12906.80585,K


Now the unlabeled data that you supplied has been labeled. It has been saved in the file 'retrained_output.csv'.